# Linear Solver Performance: Evaluation of Single Core Benchmarks

In [1]:
//#r "C:\Users\jenkinsci\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Application.SipPoisson;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
wmg.Init("WIP-k-LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'WIP-k-LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer'.


In [3]:
// Manually add database (e.g. when accessing the database of user `JenkinsCI` as some other, regular guy)
//var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

In [4]:
databases

#0: { Session Count = 242; Grid Count = 30; Path = \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer }


In [5]:
wmg.Sessions

#0: WIP-k-LinslvPerfSer	SIP_Poisson_J320_k2_direct_pardiso	05/26/2022 08:20:02	608bc9a2...
#1: WIP-k-LinslvPerfSer	XdgStokes-J13824_p3_exp_Kcycle_schwarz*	05/27/2022 10:29:20	1edf913f...
#2: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J256_p2_exp_gmres_levelpmg	05/27/2022 01:00:13	87e4f84a...
#3: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J4096_p3_exp_gmres_levelpmg	05/27/2022 01:05:15	830a6c50...
#4: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k2_direct_pardiso	05/26/2022 08:20:21	c44ba892...
#5: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J36864_p5_exp_gmres_levelpmg	05/27/2022 01:09:31	2de5e265...
#6: WIP-k-LinslvPerfSer	SIP_Poisson_J2560_k3_direct_pardiso	05/26/2022 08:21:39	8504819f...
#7: WIP-k-LinslvPerfSer	SIP_Poisson_J69120_k3_exp_Kcycle_schwarz	05/26/2022 08:29:06	c67bf727...
#8: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J2304_p2_exp_gmres_levelpmg	05/27/2022 01:01:16	d6fdea1e...
#9: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J256_p2_direct_pardiso	05/27/2022 00:52:12	8ff5081c...
#10: WIP-k-Linslv

In [6]:
var FailedSessions = wmg.Sessions.Where(Si => Si.SuccessfulTermination == false
                                              || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0);
FailedSessions

#0: WIP-k-LinslvPerfSer	XdgStokes-J13824_p3_exp_Kcycle_schwarz*	05/27/2022 10:29:20	1edf913f...
#1: WIP-k-LinslvPerfSer	XdgStokes-J4096_p5_exp_gmres_levelpmg*	05/27/2022 10:25:54	5d4f8026...
#2: WIP-k-LinslvPerfSer	XdgStokes-J4096_p5_direct_pardiso*	05/27/2022 10:21:40	086c2b7b...
#3: WIP-k-LinslvPerfSer	XdgStokes-J13824_p5_exp_gmres_levelpmg*	05/27/2022 10:26:19	0814a184...
#4: WIP-k-LinslvPerfSer	XdgStokes-J4096_p5_exp_Kcycle_schwarz*	05/27/2022 10:30:35	88527aab...
#5: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J36864_p2_pMultigrid*	05/27/2022 01:24:38	a22a446e...
#6: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J16384_p2_pMultigrid*	05/27/2022 01:24:10	41a8834f...
#7: WIP-k-LinslvPerfSer	BottiPietroStokes2D-J2304_p2_pMultigrid*	05/27/2022 01:23:16	f6154f32...
#8: WIP-k-LinslvPerfSer	XdgStokes-J32768_p3_exp_Kcycle_schwarz*	05/27/2022 10:29:46	4e14a171...
#9: WIP-k-LinslvPerfSer	XdgStokes-J13824_p5_exp_Kcycle_schwarz*	05/27/2022 10:31:00	51f275f3...


In [7]:
//NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not terminate successfully.");

In [8]:
//FailedSessions.ForEach(s => s.Delete(true));

## Create Table for Post-Processing

The timeing information that is requires for this study is not pesent in the default session table.
Instead, it must be extracted from the *session profiling*.

To obtain timing-measurements, the instrumentation generated by certain `BlockTrace`-blocks within the `UniSolver` class is extracted and added to the data table:

In [9]:
// evaluators to add additional columns to the session table
static class AddCols {
    static int GetSolType(ISessionInfo SI) {
        if(SI.GetControl().GetSolverType().Name.Contains("XNSE"))
            return 3;
        if(SI.GetControl().GetSolverType().Name.Contains("XdgPoisson3Main"))
            return 2;
        return 1;
    }
    
    
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        ilPSP.Tracing.MethodCallRecord nd;
        if(GetSolType(SI) == 3)
            nd = mcr.FindChildren("*.XdgTimestepping.ComputeOperatorMatrix").Single();
        else 
            nd  = mcr.FindChildren("MatrixAssembly").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Aggregation_basis_init").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Init").Single();
        //var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Run").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
    }
    static public object ComputeNodeName(ISessionInfo SI) {
        return SI.ComputeNodeNames.First();
    }
    
    //static public object DOFs(ISessionInfo SI) {
    //    int NoOfItr = Convert.ToInt32(SI.KeysAndQueries["NoIter"]);
    //    return NoOfItr;
    //}
    
    static public object DGdegree(ISessionInfo SI) {
        switch(GetSolType(SI)) {
            case 3:
            return SI.KeysAndQueries.Single(kv => kv.Key.Contains("DGdegree:Velocity")).Value;
            case 2:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:u").Value;    
            case 1:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:T").Value;
        }
        throw new ArgumentException();
    }
}

In [10]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
wmg.AdditionalSessionTableColums.Add("ComputeNodeName", AddCols.ComputeNodeName);
wmg.AdditionalSessionTableColums.Add("DGdegree", AddCols.DGdegree);
//wmg.AdditionalSessionTableColums.Add("DOFs", AddCols.DOFs);

In [11]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

In [12]:
//wmg.Sessions[0].KeysAndQueries["NoIter"].GetType()

In [13]:
//wmg.Sessions[0].Name

In [14]:
//SessTab.GetColumn("LinearSolver.Name")

List of all available Data Columns:

In [15]:
SessTab.GetColumnNames().ToConcatString("", "; ", "")

ProjectName; SessionName; DGdegree:T; DGdegree:Tex; Bndtype:Dirichlet; Bndtype:Neumann; TimesteppingMode; GeneratedFromCode; ControlFileText_Index; NoOfMultigridLevels; InitialValues_EvaluatorsVec[0].Key; InitialValues_Evaluators_TimeDep[0].Key; dtFixed; penalty_poisson; ExactSolution_provided; SuppressExceptionPrompt; LinearSolver.Name; LinearSolver.Shortname; LinearSolver.TestSolution; LinearSolver.WhichSolver; NonLinearSolver.verbose; NonLinearSolver.MaxSolverIterations; NonLinearSolver.MinSolverIterations; NonLinearSolver.ConvergenceCriterion; NonLinearSolver.UnderRelax; NonLinearSolver.SolverCode; NonLinearSolver.constantNewtonIterations; NonLinearSolver.HomotopyStepLongFail; NonLinearSolver.Globalization; AgglomerationThreshold; MultiStepInit; TimeSteppingScheme; LevelSet_ConvergenceCriterion; GridPartType; NoOfTimesteps; staticTimestep; Endtime; saveperiod; rollingSaves; dtMin; dtMax; ImmediatePlotPeriod; SuperSampling; savetodb; AlternateDbPaths[0].Item1; logFileDirectory; Para

Select those columns which are of interest:

In [16]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree", "Grid:NoOfCells", "LinearSolver.Name", "LinearSolver.Shortname", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter", "ComputeNodeName", "RegularTerminated");

In [17]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

Only consider runs which have been successful:

In [18]:
SubTab = SubTab.ExtractRows(delegate(int iRow, IDictionary<string, object> row) {
    return (bool)(row["RegularTerminated"]);
});

In [19]:
//var SubTab = TableExtensions.LoadFromFile("temp.json");

In [20]:
//SubTab.Print();

## Vizualisation of Results

The following data is available:

In [21]:
SubTab.GetColumnNames()

index,value
0,SessionName
1,DGdegree
2,Grid:NoOfCells
3,LinearSolver.Name
4,LinearSolver.Shortname
5,DOFs
6,MatrixAssembly
7,Aggregation_basis_init_time
8,Solver_Init_time
9,Solver_Run_time


Available DG degrees:

In [22]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree").ToSet().OrderBy(s => s).ToArray();
DGdegrees

index,value
0,2
1,3
2,5


All used solvers:

In [23]:
SubTab.GetColumn<string>("LinearSolver.Shortname").ToSet()

index,value
0,PARDISO
1,GMRES w p2G
2,OrthoMG w Add Swz
3,Ortho w pmG


In [24]:
const string Pardiso = "PARDISO";
const string OrmgSwz = "OrthoMG w Add Swz";
const string GmrsP2g = "GMRES w p2G";
const string OrmgPmg = "Ortho w pmG";
const string OrmgILU = "OrthoMG w ILU";

Used Cases:

In [25]:
const string Poisson = "SIP_Poisson";
const string XPoisson = "XdgPoisson";
const string Stokes2D = "BottiPietroStokes2D";
const string Stokes3D = "BottiPietroStokes3D";
const string XStokes = "XdgStokes";
string[] AllCases = new string[] { Poisson, XPoisson, Stokes2D, Stokes3D, XStokes };

In [26]:
string ExtractCase(string sessionName) {
    return AllCases.Single(caseName => sessionName.Contains(caseName));
}

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [27]:
PlotFormat SlvCode2Pltfmt(string solver_name, string caseName) { 
    var Fmt = new PlotFormat();
    switch(solver_name) {
        case Pardiso: 
            Fmt.PointType = PointTypes.OpenCircle;
            Fmt.DashType  = DashTypes.Dotted;
            break;
        case GmrsP2g: 
            Fmt.PointType = PointTypes.Box;
            break;
        case OrmgSwz: 
            Fmt.PointType = PointTypes.LowerTriangle;
            break;
        case OrmgILU:
            Fmt.PointType = PointTypes.Diamond;
            break;
        case OrmgPmg:
            Fmt.PointType = PointTypes.Asterisk;
            break;
        default:
            Console.WriteLine("unknown: " + solver_name); 
            break;
    } 
    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.85;
    Fmt.LineWidth = 2;    
    Fmt.Style     = Styles.LinesPoints;
    
    if(caseName.Contains(Poisson))
        Fmt.LineColor = LineColors.Green;
    else if(caseName.Contains(XPoisson))
        Fmt.LineColor = LineColors.Blue;
    else if(caseName.Contains(Stokes2D))
        Fmt.LineColor = LineColors.Orange;
    else if(caseName.Contains(XStokes))
        Fmt.LineColor = LineColors.Red;
    

    return Fmt;
}

The following function will later be used to check the regression solpe of runtimes
(ideally, one expects linear runtime of a solver with respect to degrees-of-freedom):

In [28]:
void AssertSlope(Plot2Ddata plot, string caseName, string solverName, double allowedSlope, string info) {
   NUnit.Framework.Assert.LessOrEqual(
      plot.Regression().Single(tt =>  tt.Key.Contains(caseName) && tt.Key.Contains(solverName)).Value,
      allowedSlope,
      $"{solverName}/{info} scaling for {caseName} exceeds limit");
}

In [29]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax, double LegendYpos) {

int rows    = DGdegrees.Length;
int columns = 1;
string[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows + 1,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           string solver_name = (string) (Row["LinearSolver.Shortname"]);
           //ignore the solvers specified in ingore_solvers
           foreach(string sc in ignore_solvers){
               if(solver_name == sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           
           //process the other solvers
           string caseName = ExtractCase( (string) Row["SessionName"]);
           Fmt = SlvCode2Pltfmt(solver_name, caseName);
           Nmn = solver_name + "/" + caseName;
       });
    
       double[] dof = new[]{1e3,1e6};
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e7;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 5;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == rows - 1) {
          multiplots[iRow,iCol].ShowLegend = true;
          //multiplots[iRow,iCol].LegendAlignment = new string[]{"o", "r", "t" };
          multiplots[iRow,iCol].LegendFont = 12;
          multiplots[iRow,iCol].Legend_maxrows = 100;
           multiplots[iRow,iCol].LegendPosition = new double[] { 4e7, LegendYpos };
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

### Total Runtime

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [30]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4, 1e-5);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 PARDISO/SIP P oisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 Ortho w pmG/SIP P oisson 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear

In [31]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4, 1e-5);
multiplots.PlotCairolatex().SaveTo("Runtime.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [32]:
/*var legend = new Plot2Ddata();
legend.AddDataGroup("Pardiso",new double[] {1,2},new double[] {1,2}, "-og");
legend.ShowXtics = false;
legend.ShowXtics = false;
//legend.XrangeMin = 3;
//legend.XrangeMax = 3.1;

//legend.AddGnuplotCommand("set border 0");
//legend.AddGnuplotCommand("set style line 101 lc rgb '#808080' lt 1 lw 1");
legend.AddGnuplotCommand("unset xlabel");
legend.AddGnuplotCommand("unset ylabel");
legend.AddGnuplotCommand("set format x ''");
legend.AddGnuplotCommand("set format y ''");
legend.AddGnuplotCommand("set tics scale 0");

legend.PlotNow()*/

#### Regression and Error checking (Runtime)

For polynomial degree 2:

In [33]:
multiplots[0,0].Regression()

index,Key,Value
0,PARDISO/SIP_Poisson,1.362193952828686
1,GMRES w p2G/BottiPietroStokes2D,0.8255318011141116
2,PARDISO/BottiPietroStokes2D,0.8614271930451801
3,GMRES w p2G/XdgStokes,1.3386831611527736
4,OrthoMG w Add Swz/SIP_Poisson,0.9821859823116871
5,OrthoMG w Add Swz/BottiPietroStokes2D,1.1134131393577704
6,GMRES w p2G/SIP_Poisson,0.9718012377428541
7,OrthoMG w Add Swz/XdgStokes,1.407526940314998
8,PARDISO/XdgStokes,2.099653927493228
9,Ortho w pmG/SIP_Poisson,1.0380875734071495


For polynomial degree 3:

In [34]:
multiplots[1,0].Regression()

index,Key,Value
0,GMRES w p2G/BottiPietroStokes2D,0.9789867618769059
1,PARDISO/SIP_Poisson,1.327703472826657
2,OrthoMG w Add Swz/SIP_Poisson,1.0777122020057617
3,OrthoMG w Add Swz/BottiPietroStokes2D,0.9954313387713271
4,PARDISO/BottiPietroStokes2D,0.8834586897990597
5,PARDISO/XdgPoisson,1.1582655713550658
6,GMRES w p2G/SIP_Poisson,0.9964878207693134
7,Ortho w pmG/SIP_Poisson,0.9398673097196037
8,GMRES w p2G/XdgStokes,1.101322184123572
9,Ortho w pmG/BottiPietroStokes2D,1.2760017996330488


For polynomial degree 5:

In [35]:
multiplots[2,0].Regression()

index,Key,Value
0,GMRES w p2G/BottiPietroStokes2D,1.108048197116999
1,PARDISO/SIP_Poisson,1.6907184914675593
2,Ortho w pmG/SIP_Poisson,1.1498319868051075
3,PARDISO/BottiPietroStokes2D,1.1427522082636123
4,OrthoMG w Add Swz/BottiPietroStokes2D,1.1589586891185335
5,GMRES w p2G/SIP_Poisson,0.9598007611890037
6,OrthoMG w Add Swz/SIP_Poisson,0.9878264378493566
7,Ortho w pmG/BottiPietroStokes2D,1.2964709501244012
8,PARDISO/XdgStokes,NaN
9,OrthoMG w Add Swz/XdgStokes,NaN


In [36]:
AssertSlope(multiplots[0,0], Poisson, GmrsP2g, 1.14, "k=2");
AssertSlope(multiplots[1,0], Poisson, GmrsP2g, 1.1, "k=3");
AssertSlope(multiplots[2,0], Poisson, GmrsP2g, 1.1, "k=5");

Experimental: Latex Preview:

In [37]:
//var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
//multiplots.PlotCairolatex().PlotNow()

### Number of Iterations

For the iterative solvers, obe would expect **almost constant number of solver iterations**, 
independent of grid resolution. It may, however depend in nonlinar fashion on the DG polynomial order.

In [38]:
var multiplots = PlotSolverBehave("NoIter", false, 0, 300, -150);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 PARDISO/SIP P oisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 Ortho w pmG/SIP P oisson 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear

The following function will later be used to check the regression solpe of runtimes
(ideally, one expects constant number of iterations of a multigrid-solver with respect to degrees-of-freedom):

In [39]:
void AssertLimit(Plot2Ddata plot, string caseName, string solverName, double allowedLimit, string info) {
   NUnit.Framework.Assert.LessOrEqual(
      plot.dataGroups.Single(tt =>  tt.Name.Contains(caseName) && tt.Name.Contains(solverName)).Values.Max(),
      allowedLimit,
      $"{solverName}/{info} scaling for {caseName} exceeds limit");
}


In [40]:
//multiplots[0,0].dataGroups

In [41]:
AssertLimit(multiplots[0,0], Poisson,  GmrsP2g, 130, "k=2");
AssertLimit(multiplots[0,0], XPoisson, GmrsP2g, 70, "k=2");
AssertLimit(multiplots[0,0], Stokes2D, GmrsP2g, 80, "k=2");

AssertLimit(multiplots[0,0], Poisson,  OrmgSwz, 35, "k=2");
AssertLimit(multiplots[0,0], XPoisson, OrmgSwz, 30, "k=2");
//AssertLimit(multiplots[0,0], Stokes2D, OrmgSwz, 80, "k=2");  // not working yet
//AssertLimit(multiplots[0,0], XStokes,  OrmgSwz, 130, "k=2"); // not working yet

AssertLimit(multiplots[0,0], Poisson,  OrmgPmg, 35, "k=2");
AssertLimit(multiplots[0,0], XPoisson, OrmgPmg, 60, "k=2"); // not really ideal
//AssertLimit(multiplots[0,0], Stokes2D, OrmgPmg, 80, "k=2");  // not working yet
//AssertLimit(multiplots[0,0], XStokes,  OrmgPmg, 130, "k=2"); // not working yet

AssertLimit(multiplots[0,0], Poisson,  Pardiso, 1, "k=2");
AssertLimit(multiplots[0,0], XPoisson, Pardiso, 1, "k=2");
AssertLimit(multiplots[0,0], Stokes2D, Pardiso, 1, "k=2"); 
AssertLimit(multiplots[0,0], XStokes,  Pardiso, 1, "k=2");

In [42]:
multiplots[1,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464, 1703936 ]","[ 137, 147, 155, 175, 195, 203, 213, 219, 223 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/SIP_Poisson,False,False,"[ 6400, 51200, 409600 ]","[ 1, 1, 1 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/SIP_Poisson,False,False,"[ 6400, 51200, 409600, 1382400 ]","[ 1, 5, 8, 9 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464, 1703936 ]","[ 1, 1, 1, 5, 7, 7, 8, 12, 8 ]"
4,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464 ]","[ 1, 1, 1, 1, 1, 1, 1, 1 ]"
5,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgPoisson,False,False,"[ 320, 1760, 11840, 87680, 678080 ]","[ 1, 1, 1, 1, 1 ]"
6,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/SIP_Poisson,False,False,"[ 6400, 51200, 409600, 1382400 ]","[ 72, 71, 141, 145 ]"
7,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/SIP_Poisson,False,False,"[ 6400, 51200, 409600, 1382400 ]","[ 4, 4, 4, 4 ]"
8,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/XdgStokes,False,False,"[ 37520, 293440, 986160, 2324000 ]","[ 283, 335, 337, 361 ]"
9,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464, 1703936 ]","[ 9, 9, 7, 7, 11, 11, 13, 14, 16 ]"


In [43]:
AssertLimit(multiplots[1,0], Poisson,  GmrsP2g, 150, "k=3");  // not really ideal
AssertLimit(multiplots[1,0], XPoisson, GmrsP2g, 120, "k=3");
AssertLimit(multiplots[1,0], Stokes2D, GmrsP2g, 230, "k=3"); // not really ideal

AssertLimit(multiplots[1,0], Poisson,  OrmgSwz, 35, "k=3");
AssertLimit(multiplots[1,0], XPoisson, OrmgSwz, 30, "k=3");
//AssertLimit(multiplots[1,0], Stokes2D, OrmgSwz, 80, "k=3");  // not working yet
//AssertLimit(multiplots[1,0], XStokes,  OrmgSwz, 130, "k=3"); // not working yet

AssertLimit(multiplots[1,0], Poisson,  OrmgPmg, 10, "k=3");
AssertLimit(multiplots[1,0], XPoisson, OrmgPmg, 20, "k=3");
AssertLimit(multiplots[1,0], Stokes2D, OrmgPmg, 20, "k=3");
//AssertLimit(multiplots[1,0], XStokes,  OrmgPmg, 1, "k=3"); // not working

AssertLimit(multiplots[1,0], Poisson,  Pardiso, 1, "k=3");
AssertLimit(multiplots[1,0], XPoisson, Pardiso, 1, "k=3");
AssertLimit(multiplots[1,0], Stokes2D, Pardiso, 1, "k=3"); 
AssertLimit(multiplots[1,0], XStokes,  Pardiso, 1, "k=3");

In [44]:
multiplots[2,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472, 933888, 2101248 ]","[ 238, 275, 301, 343, 422, 459, 524, 518 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/SIP_Poisson,False,False,"[ 17920, 143360 ]","[ 1, 1 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/SIP_Poisson,False,False,"[ 17920, 143360, 1146880 ]","[ 3, 3, 3 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472, 933888 ]","[ 1, 1, 1, 1, 1, 1, 1 ]"
4,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472, 933888, 2101248 ]","[ 1, 1, 4, 3, 6, 7, 10, 12 ]"
5,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/SIP_Poisson,False,False,"[ 17920, 143360, 1146880 ]","[ 154, 171, 182 ]"
6,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/SIP_Poisson,False,False,"[ 17920, 143360, 1146880 ]","[ 1, 4, 5 ]"
7,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472, 933888, 2101248 ]","[ 6, 6, 6, 7, 7, 8, 10, 11 ]"
8,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgStokes,False,False,[ 108808 ],[ 1 ]
9,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgStokes,False,False,[ 108808 ],[ 15 ]


In [45]:
AssertLimit(multiplots[2,0], Poisson,  GmrsP2g, 200, "k=5");
AssertLimit(multiplots[2,0], XPoisson, GmrsP2g, 200, "k=5");
//AssertLimit(multiplots[2,0], Stokes2D, GmrsP2g, 230, "k=5");  // not working yet

AssertLimit(multiplots[2,0], Poisson,  OrmgSwz, 35, "k=5");
AssertLimit(multiplots[2,0], XPoisson, OrmgSwz, 35, "k=5");
//AssertLimit(multiplots[2,0], Stokes2D, OrmgSwz, 80, "k=5"); 
AssertLimit(multiplots[2,0], XStokes,  OrmgSwz, 70, "k=5"); // not working yet

AssertLimit(multiplots[2,0], Poisson,  OrmgPmg, 10, "k=5");
AssertLimit(multiplots[2,0], XPoisson, OrmgPmg, 45, "k=5");
AssertLimit(multiplots[2,0], Stokes2D, OrmgPmg, 20, "k=5");
//AssertLimit(multiplots[2,0], XStokes,  OrmgPmg, 1, "k=5"); // not working

AssertLimit(multiplots[2,0], Poisson,  Pardiso, 1, "k=5");
AssertLimit(multiplots[2,0], XPoisson, Pardiso, 1, "k=5");
AssertLimit(multiplots[2,0], Stokes2D, Pardiso, 1, "k=5"); 
AssertLimit(multiplots[2,0], XStokes,  Pardiso, 1, "k=5");

### Runtime per Iteration

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [46]:
var multiplots = PlotSolverBehave("Solver_TimePerIter", true, 1e-2, 1e+6, 1e-3);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 PARDISO/SIP P oisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 Ortho w pmG/SIP P oisson 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 


In [47]:
multiplots[0,0].Regression()

index,Key,Value
0,PARDISO/SIP_Poisson,1.417759821056381
1,GMRES w p2G/BottiPietroStokes2D,0.8174335985283153
2,PARDISO/BottiPietroStokes2D,0.8823369853670304
3,GMRES w p2G/XdgStokes,1.2775983612117026
4,OrthoMG w Add Swz/SIP_Poisson,0.6605729223874724
5,OrthoMG w Add Swz/BottiPietroStokes2D,0.7182320328406935
6,GMRES w p2G/SIP_Poisson,0.8138604477043927
7,OrthoMG w Add Swz/XdgStokes,0.47642522155132544
8,PARDISO/XdgStokes,2.1560203865301113
9,Ortho w pmG/SIP_Poisson,0.9701340323289606


In [48]:
multiplots[1,0].Regression()

index,Key,Value
0,GMRES w p2G/BottiPietroStokes2D,0.9185241253200137
1,PARDISO/SIP_Poisson,1.3665966955248112
2,OrthoMG w Add Swz/SIP_Poisson,0.734429674340111
3,OrthoMG w Add Swz/BottiPietroStokes2D,0.6157448185796863
4,PARDISO/BottiPietroStokes2D,0.9016938037914177
5,PARDISO/XdgPoisson,1.1937246070701404
6,GMRES w p2G/SIP_Poisson,0.8653799030061212
7,Ortho w pmG/SIP_Poisson,0.9729263846595524
8,GMRES w p2G/XdgStokes,1.0582624804853726
9,Ortho w pmG/BottiPietroStokes2D,1.1802984649470836


In [49]:
multiplots[2,0].Regression()

index,Key,Value
0,GMRES w p2G/BottiPietroStokes2D,0.9936531380937801
1,PARDISO/SIP_Poisson,1.8252014052248178
2,Ortho w pmG/SIP_Poisson,1.2303395175502363
3,PARDISO/BottiPietroStokes2D,1.2101723720290958
4,OrthoMG w Add Swz/BottiPietroStokes2D,0.7886168918678776
5,GMRES w p2G/SIP_Poisson,0.9398166158353835
6,OrthoMG w Add Swz/SIP_Poisson,0.6965324624097373
7,Ortho w pmG/BottiPietroStokes2D,1.2047066373932198
8,PARDISO/XdgStokes,NaN
9,OrthoMG w Add Swz/XdgStokes,NaN


In [50]:
AssertSlope(multiplots[0,0], Poisson, GmrsP2g, 1.1, "k=2");
AssertSlope(multiplots[1,0], Poisson, GmrsP2g, 1.1, "k=3");
AssertSlope(multiplots[2,0], Poisson, GmrsP2g, 1.1, "k=5");

## Appendix: Table Form of Extracted Data

In [51]:
SubTab.Print()

     SessionName                                       DGdegree Grid:NoOfCells LinearSolver.Name                                          LinearSolver.Shortname DOFs    MatrixAssembly Aggregation_basis_init_time Solver_Init_time Solver_Run_time Solver_InitAndRun_time NoIter Solver_TimePerIter   ComputeNodeName                               RegularTerminated 
0:   SIP_Poisson_J320_k2_direct_pardiso                2        320            Sparse direct solver PARDISO                               PARDISO                3200    0.5103087      0.070728                    0.0795447        0.4002311       0.5505038              1      0.4002311            hpccluster2.fdy.maschinenbau.tu-darmstadt.de  True              
1:   BottiPietroStokes2D-J256_p2_exp_gmres_levelpmg    2        256            GMRES with p-two-grid preconditioner                       GMRES w p2G            3840    1.0358754      0.0663225                   0.3970185        2.0419999       2.5053409              70     0.0

21:  BottiPietroStokes2D-J576_p2_direct_pardiso        2        576            Sparse direct solver PARDISO                               PARDISO                8640    1.0662306      0.0819303                   0.3459209        0.7298238       1.157675               1      0.7298238            fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
22:  BottiPietroStokes2D-J65536_p3_exp_Kcycle_schwarz  3        65536          OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      1703936 71.4558552     6.508481                    103.3438377      350.5737693     460.426088             8      43.8217211625        fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
23:  BottiPietroStokes2D-J36864_p3_exp_Kcycle_schwarz  3        36864          OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      958464  50.1817745     3.5554097                   68.3035279       699.8179537     771.6768913            12     58.

43:  SIP_Poisson_J2560_k5_exp_Kcycle_schwarz           5        2560           OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      143360  60.9176937     15.2694755                  9.2438828        72.3660612      96.87941950000001      4      18.0915153           hpccluster2.fdy.maschinenbau.tu-darmstadt.de  True              
44:  BottiPietroStokes2D-J147456_p2_exp_Kcycle_schwarz 2        147456         OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      2211840 99.167531      11.7500272                  139.422806       11554.1446998   11705.317533           20     577.70723499         fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
45:  SIP_Poisson_J163840_k2_exp_Kcycle_schwarz         2        163840         OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      1638400 114.9212521    14.6022288                  75.4499166       422.5124921     512.5646375            8      52.

65:  BottiPietroStokes2D-J36864_p2_exp_gmres_levelpmg  2        36864          GMRES with p-two-grid preconditioner                       GMRES w p2G            552960  26.4585914     2.0930409                   23.0713838       77.6286212      102.7930459            67     1.1586361373134328   fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
66:  BottiPietroStokes2D-J16384_p2_exp_gmres_levelpmg  2        16384          GMRES with p-two-grid preconditioner                       GMRES w p2G            245760  14.4869606     1.0649595                   12.3585403       34.9724618      48.3959616             69     0.506847272463768    fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
67:  BottiPietroStokes2D-J1024_p3_direct_pardiso       3        1024           Sparse direct solver PARDISO                               PARDISO                26624   2.5772847      0.1124682                   1.0588977        2.8039785       3.9753444              1      2.8

87:  BottiPietroStokes2D-J256_p3_exp_Kcycle_schwarz    3        256            OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      6656    1.0733077      0.0745172                   0.408717         1.3068677       1.7901019              1      1.3068677            fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
88:  BottiPietroStokes2D-J36864_p3_direct_pardiso      3        36864          Sparse direct solver PARDISO                               PARDISO                958464  40.5784859     3.0369611                   31.0931285       270.5187546     304.64884420000004     1      270.5187546          fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
89:  BottiPietroStokes2D-J2304_p2_exp_Kcycle_schwarz   2        2304           OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      34560   3.2965522      0.1945325                   2.1983372        4.2381305       6.6310002              9      0.4

109: XdgStokes-J13824_p2_exp_Kcycle_schwarz            2        13824          OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      478992  231.3017373    1.6880138                   41.2329588       1907.9247531    1950.8457257           60     31.798745885000002   fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
110: XdgStokes-J512_p5_exp_gmres_levelpmg              5        512            GMRES with p-two-grid preconditioner                       GMRES w p2G            108808  334.6528356    3.4998463                   29.7080281       403.708847      436.9167214            583    0.6924680051457975   fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
111: XdgStokes-J512_p2_exp_gmres_levelpmg              2        512            GMRES with p-two-grid preconditioner                       GMRES w p2G            18224   9.9959225      0.0936759                   1.1462553        6.2359868       7.475918               144    0.0

131: SIP_Poisson_J2560_k2_exp_gmres_levelpmg           2        2560           GMRES with p-two-grid preconditioner                       GMRES w p2G            25600   1.794755       0.2278596                   0.4592858        3.4678657       4.1550111              50     0.069357314          hpccluster2.fdy.maschinenbau.tu-darmstadt.de  True              
132: XdgStokes-J512_p2_exp_Kcycle_schwarz              2        512            OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      18224   9.2955005      0.0815526                   1.0369425        5.1846578       6.303152900000001      1      5.1846578            fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
133: BottiPietroStokes2D-J4096_p5_pMultigrid           5        4096           Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            233472  17.6280759     1.4545393                   29.8825617       1431.3122242    1462.6493252           8      178

153: XdgPoisson-J512_k2_direct_pardiso                 2        512            Sparse direct solver PARDISO                               PARDISO                5920    1.6831624      0.0663439                   0.1484536        0.7126853       0.9274827999999999     1      0.7126853            hpccluster2.fdy.maschinenbau.tu-darmstadt.de  True              
154: XdgStokes-J512_p3_exp_Kcycle_schwarz              3        512            OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      37520   27.2814997     0.1978566                   5.0418134        27.3092004      32.5488704             16     1.706825025          fdygitrunner.fdy.maschinenbau.tu-darmstadt.de True              
155: BottiPietroStokes2D-J16384_p3_pMultigrid          3        16384          Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            425984  17.0964669     1.6492425                   23.6935646       4469.510126     4494.8529331           13     343

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

